In [1]:
import sqlite3
import numpy as np
import math
from sklearn.model_selection import train_test_split
conn = sqlite3.connect('trunc_database05192019.sqlite3')
c = conn.cursor()
c.execute('SELECT DISTINCT standard_inchi,protein_sequence,standard_value FROM interactions WHERE protein_sequence = (SELECT protein_sequence FROM interactions WHERE ROWID = 5000)')
d1 = c.fetchall()
c.execute('SELECT DISTINCT standard_inchi,protein_sequence,standard_value FROM interactions WHERE protein_sequence <> (SELECT protein_sequence FROM interactions WHERE ROWID = 5000)')
d2 = c.fetchall()
conn.close()

In [2]:
din1=[[0] for x in range(len(d1))]
dpr1=[[0] for x in range(len(d1))]
dca1=[[0] for x in range(len(d1))]
i=0
for row in d1:
    din1[i] = [ord(c) for c in d1[i][0]] # inchis
    dpr1[i] = [ord(c) for c in d1[i][1]] # proteins
    if d1[i][2]=='': dca1[i]=0
    elif float(d1[i][2])>100: dca1[i] = 0 # categories - can adjust this threshold for better results
    else: dca1[i] = 1   
    i += 1

In [3]:
din2=[[0] for x in range(len(d2))]
dpr2=[[0] for x in range(len(d2))]
dca2=[[0] for x in range(len(d2))]
i=0
for row in d2:
    din2[i] = [ord(c) for c in d2[i][0]] # inchis
    dpr2[i] = [ord(c) for c in d2[i][1]] # proteins
    if d2[i][2]=='': dca2[i]=0
    elif float(d2[i][2])>100: dca2[i] = 0 # categories - can adjust this threshold for better results
    else: dca2[i] = 1   
    i += 1

In [4]:
maxlen=0
for row in range(len(din1)):
    maxlen = len(din1[row]) if len(din1[row])>maxlen else maxlen
for row in range(len(din2)):
    maxlen = len(din2[row]) if len(din2[row])>maxlen else maxlen
for row in range(len(din1)):
    din1[row].extend([0 for x in range(maxlen-len(din1[row]))])
for row in range(len(din2)):
    din2[row].extend([0 for x in range(maxlen-len(din2[row]))])
maxlen=0
for row in range(len(dpr1)):
    maxlen = len(dpr1[row]) if len(dpr1[row])>maxlen else maxlen
for row in range(len(dpr2)):
    maxlen = len(dpr2[row]) if len(dpr2[row])>maxlen else maxlen
for row in range(len(dpr1)):
    dpr1[row].extend([0 for x in range(maxlen-len(dpr1[row]))])
for row in range(len(dpr2)):
    dpr2[row].extend([0 for x in range(maxlen-len(dpr2[row]))])
    
nin1 = np.array(din1).astype(float) # turn into Numpy Array for Keras - INICHIS (samples x asciis)
npr1 = np.array(dpr1).astype(float) # turn into Numpy Array for Keras - PROTEINS (samples x asciis)
nca1 = np.array(dca1).astype(float) # turn into Numpy Array for Keras - IC50s (samples x 0/1)

nin2 = np.array(din2).astype(float) # turn into Numpy Array for Keras - INICHIS (samples x asciis)
npr2 = np.array(dpr2).astype(float) # turn into Numpy Array for Keras - PROTEINS (samples x asciis)
nca2 = np.array(dca2).astype(float) # turn into Numpy Array for Keras - IC50s (samples x 0/1)

In [6]:
maxval = 0
for row in range(len(nin1)):
    maxval = max(nin1[row]) if max(nin1[row])>maxval else maxval
for row in range(len(nin2)):
    maxval = max(nin2[row]) if max(nin2[row])>maxval else maxval
for row in range(len(nin1)):
    nin1[row] = np.divide(nin1[row],float(maxval))
for row in range(len(nin2)):
    nin2[row] = np.divide(nin2[row],float(maxval))
maxval = 0
for row in range(len(npr1)):
    maxval = max(npr1[row]) if max(npr1[row])>maxval else maxval
for row in range(len(npr2)):
    maxval = max(npr2[row]) if max(npr2[row])>maxval else maxval
for row in range(len(npr1)):
    npr1[row] = np.divide(npr1[row],float(maxval))
for row in range(len(npr2)):
    npr2[row] = np.divide(npr2[row],float(maxval))

In [5]:
print (len(nin2))
print (len(npr2))

599745
599745


In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [8]:

X_test = np.concatenate((nin1,npr1),axis=1)
X_train = np.concatenate((nin2,npr2),axis=1)
y_test = nca1
y_train = nca2

#X = np.concatenate((nin2,npr2),axis=1)
#X_train, X_test, y_train, y_test = train_test_split(X, nca2, test_size=0.2, random_state=61)
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# one hot encode outputs - needed for Keras optimizer
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

MemoryError: 

In [13]:

print(type(nin2))
print(nin2.shape)
print(npr2.shape)


<class 'numpy.ndarray'>
(599745, 351)
(599745, 1453)
